# Taking a Look at How True the Famous Saying: 

*"Offense Wins Games, Defense Wins Championships"* - Paul "Bear" Bryant

Methodology: 

Investigaet the correlation between offensive rating/defensive rating and win percentage in the regular season and playoffs

Data:
- Past 5 years of regular season data for each team (W, L, W_PCT, E_OFF_RATING, E_DEF_RATING, E_NET_RATING)
- Past 5 years of postseason data for each team (W, L, W_PCT, E_OFF_RATING, E_DEF_RATING, E_NET_RATING)

In [1]:
import pandas as pd
import numpy as np
import time

from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder, teamestimatedmetrics, teamdashboardbygeneralsplits

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
nba_teams = teams.get_teams()

In [4]:
hawks_id = nba_teams[0]['id']

In [5]:
games_23_dict = {}

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = hawks_id)
all_games = gamefinder.get_data_frames()[0]
games_23 = all_games[all_games.SEASON_ID.str[-4:] == '2022']
games_23_dict[nba_teams[0]['full_name']] = games_23

In [34]:
games_23_dict

{'Atlanta Hawks':    SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
 0      22022  1610612737               ATL  Atlanta Hawks  0022200974   
 1      22022  1610612737               ATL  Atlanta Hawks  0022200958   
 2      22022  1610612737               ATL  Atlanta Hawks  0022200947   
 3      22022  1610612737               ATL  Atlanta Hawks  0022200923   
 4      22022  1610612737               ATL  Atlanta Hawks  0022200911   
 ..       ...         ...               ...            ...         ...   
 70     22022  1610612737               ATL  Atlanta Hawks  1522200046   
 71     22022  1610612737               ATL  Atlanta Hawks  1522200034   
 72     22022  1610612737               ATL  Atlanta Hawks  1522200034   
 73     22022  1610612737               ATL  Atlanta Hawks  1522200026   
 74     22022  1610612737               ATL  Atlanta Hawks  1522200014   
 
      GAME_DATE      MATCHUP    WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  \
 0   2023-03-06

In [33]:
hawks_past5 = {}
season_ids = ['22022', '22021', '22019', '22018', '22017']
past5_games = all_games[all_games.SEASON_ID.isin(season_ids)]
past5_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612737,ATL,Atlanta Hawks,0022200974,2023-03-06,ATL @ MIA,L,239,128,...,0.739,12,25,37,28,5.0,0,10,25,-2.0
1,22022,1610612737,ATL,Atlanta Hawks,0022200958,2023-03-04,ATL @ MIA,L,239,109,...,0.867,10,27,37,22,12.0,1,16,23,-8.0
2,22022,1610612737,ATL,Atlanta Hawks,0022200947,2023-03-03,ATL vs. POR,W,239,129,...,0.813,10,33,43,30,10.0,5,17,19,18.0
3,22022,1610612737,ATL,Atlanta Hawks,0022200923,2023-02-28,ATL vs. WAS,L,239,116,...,0.895,13,29,42,21,4.0,7,9,17,-3.0
4,22022,1610612737,ATL,Atlanta Hawks,0022200911,2023-02-26,ATL vs. BKN,W,240,129,...,0.821,13,33,46,20,7.0,2,15,18,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,22017,1610612737,ATL,Atlanta Hawks,1521700053,2017-07-14,ATL vs. HOU,W,199,92,...,0.600,14,27,41,21,10.0,6,14,17,6.0
529,22017,1610612737,ATL,Atlanta Hawks,1521700037,2017-07-12,ATL vs. NOP,L,200,95,...,0.769,9,20,29,13,9.0,3,12,25,-10.0
530,22017,1610612737,ATL,Atlanta Hawks,1521700023,2017-07-10,ATL @ CHI,W,199,75,...,0.765,16,36,52,17,9.0,5,13,22,20.0
531,22017,1610612737,ATL,Atlanta Hawks,1521700015,2017-07-09,ATL @ NOP,W,201,84,...,0.679,11,23,34,14,10.0,2,14,16,2.0


In [17]:
all_games.SEASON_ID.unique()
season_ids = ['22022', '22021', '22019', '22018', '22017']

array(['22022', '12022', '42021', '52021', '22021', '12021', '42020',
       '22020', '12020', '22019', '12019', '22018', '12018', '22017',
       '12017', '42016', '22016', '12016', '42015', '22015', '12015',
       '42014', '22014', '12014', '42013', '22013', '12013', '42012',
       '22012', '12012', '42011', '22011', '12011', '42010', '22010',
       '12010', '42009', '22009', '12009', '42008', '22008', '12008',
       '42007', '22007', '12007', '22006', '12006', '22005', '12005',
       '22004', '12004', '22003', '12003', '22002', '22001', '22000',
       '21999', '41998', '21998', '41997', '21997', '41996', '21996',
       '41995', '21995', '41994', '21994', '41993', '21993', '41992',
       '21992', '21991', '41990', '21990', '21989', '41988', '21988',
       '41987', '21987', '41986', '21986', '41985', '21985', '21984',
       '41983', '21983'], dtype=object)